# Blockhash Oracle Test Script

This notebook tests the deployed oracle system by sending messages and requesting block hashes.

## 1. Configuration

In [ ]:
# Network type selection
NETWORK_TYPE = "testnets"  # "testnets" or "mainnets"

# Test configuration
# Which chain to use for read tests
READ_TEST_CHAIN = "base-sepolia"

# Whether to test broadcasting
TEST_BROADCAST = True

## 2. Initialize Environment

In [ ]:
import json
import os
import sys
import logging
import subprocess
import time
from pathlib import Path

from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3

# Add parent directory to path for imports
sys.path.append(str(Path().resolve().parent))
from ABIs import endpointV2_abi
from LZMetadata import LZMetadata
from utils import encode_headers

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

# Constants
READ_CHANNEL_ID = 4294967295  # max uint32

## 3. Helper Functions

In [ ]:
def get_vyper_abi(filepath):
    """Get ABI from Vyper contract file"""
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


def send_tx_single(w3, func, acc, value=0, gas=0):
    """Send a single transaction"""
    tx = func.build_transaction(
        {
            "from": acc.address,
            "nonce": w3.eth.get_transaction_count(acc.address),
            "value": value,
            "gas": gas,
        }
    )
    if gas > 0:
        tx["gas"] = gas
    else:
        tx["gas"] = int(w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=acc.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash

## 4. Load Deployment and Configuration

In [ ]:
# Load deployments
with open("deployments.json", "r") as f:
    deployments = json.load(f)

# Get latest deployment for network type
deployment = None
for d in reversed(deployments["deployments"]):
    if d["network_type"] == NETWORK_TYPE:
        deployment = d
        break

if not deployment:
    raise ValueError(f"No deployment found for {NETWORK_TYPE}")

logging.info(f"Using deployment from {deployment['timestamp']}")

# Load chains configuration
with open("chains.json", "r") as f:
    chains_config = json.load(f)

chains = chains_config[NETWORK_TYPE]

# Find main chain
main_chain = None
for chain_name, config in chains.items():
    if config.get("is_main_chain", False):
        main_chain = chain_name
        break

if not main_chain:
    raise ValueError(f"No main chain defined for {NETWORK_TYPE}")

logging.info(f"Main chain: {main_chain}")

## 5. Setup Account

In [ ]:
# Get account
if NETWORK_TYPE == "testnets":
    private_key = os.environ.get("WEB3_TESTNET_PK")
    if not private_key:
        raise ValueError("WEB3_TESTNET_PK not found in environment")
    account = Account.from_key(private_key)
else:
    # For mainnets, use secure key utilities
    sys.path.append(os.path.expanduser("~/projects/keys/scripts"))
    from secure_key_utils import get_account_pk

    account = get_account_pk()

logging.info(f"Account address: {account.address}")

## 6. Initialize State with Deployed Contracts

In [ ]:
# Initialize LayerZero metadata
lz = LZMetadata()

# Initialize state dictionary
state_dict = {}

# Get ABIs
ABI_RELAY = get_vyper_abi("../../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../../contracts/BlockOracle.vy")
ABI_HEADER_VERIFIER = get_vyper_abi("../../contracts/HeaderVerifier.vy")
ABI_VIEW = get_vyper_abi("../../contracts/MainnetBlockView.vy")

for chain_name, config in chains.items():
    if chain_name not in deployment["contracts"]:
        logging.warning(f"No contracts deployed on {chain_name}, skipping")
        continue

    state_dict[chain_name] = {}

    # Build RPC URL
    if "rpc" in config:
        rpc_url = config["rpc"]
    else:
        api_key = os.environ.get(config["rpc_env_var"])
        if not api_key:
            raise ValueError(f"{config['rpc_env_var']} not found in environment")
        rpc_url = config["rpc_template"].format(api_key)

    # Setup state
    state_dict[chain_name]["config"] = config
    state_dict[chain_name]["rpc"] = rpc_url
    state_dict[chain_name]["w3"] = Web3(Web3.HTTPProvider(rpc_url))

    # Get LayerZero metadata
    try:
        lz_metadata = lz.get_chain_metadata(chain_name)
        state_dict[chain_name]["eid"] = lz_metadata["metadata"]["eid"]
        state_dict[chain_name]["endpoint"] = lz_metadata["metadata"]["endpointV2"]
        state_dict[chain_name]["read_lib"] = lz_metadata["metadata"].get(
            "readLib1002", "unavailable"
        )
    except Exception as e:
        logging.warning(f"Failed to get LZ metadata for {chain_name}: {e}")

    # Load deployed contracts
    contracts = deployment["contracts"][chain_name]

    # Setup Web3 contracts
    if chain_name == main_chain:
        state_dict[chain_name]["oracle_w3"] = state_dict[chain_name]["w3"].eth.contract(
            address=contracts["MainnetBlockView"], abi=ABI_VIEW
        )
    else:
        state_dict[chain_name]["oracle_w3"] = state_dict[chain_name]["w3"].eth.contract(
            address=contracts["BlockOracle"], abi=ABI_ORACLE
        )
        state_dict[chain_name]["header_verifier_w3"] = state_dict[chain_name]["w3"].eth.contract(
            address=contracts["HeaderVerifier"], abi=ABI_HEADER_VERIFIER
        )
        state_dict[chain_name]["block_relay_w3"] = state_dict[chain_name]["w3"].eth.contract(
            address=contracts["LZBlockRelay"], abi=ABI_RELAY
        )
        state_dict[chain_name]["endpoint_w3"] = state_dict[chain_name]["w3"].eth.contract(
            address=state_dict[chain_name]["endpoint"], abi=endpointV2_abi
        )

logging.info("Loaded all deployed contracts")

## 7. Check Current Oracle State

In [ ]:
# Check oracle data on all chains
print("\n" + "=" * 80)
print("CURRENT ORACLE STATE")
print("=" * 80)

# Get main chain block number
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"\nCurrent block on {main_chain}: {main_block}")

# Check oracle state on all other chains
for chain_name in state_dict.keys():
    if chain_name == main_chain:
        continue

    oracle_w3 = state_dict[chain_name]["oracle_w3"]

    try:
        last_block = oracle_w3.functions.last_confirmed_block_number().call()
        if last_block > 0:
            block_hash = oracle_w3.functions.get_block_hash(last_block).call()
            print(f"\n{chain_name}:")
            print(f"  Last confirmed block: {last_block}")
            print(f"  Block hash: {block_hash.hex()}")
            print(f"  Blocks behind: {main_block - last_block}")
        else:
            print(f"\n{chain_name}: No blocks confirmed yet")
    except Exception as e:
        print(f"\n{chain_name}: Error reading oracle state - {e}")

## 8. Test Simple Read (Without Broadcast)

In [ ]:
# Test read on configured chain
if READ_TEST_CHAIN not in state_dict:
    print(f"Read test chain {READ_TEST_CHAIN} not found in deployment")
elif state_dict[READ_TEST_CHAIN]["read_lib"] == "unavailable":
    print(f"{READ_TEST_CHAIN} does not have read capability")
else:
    print(f"\nTesting read on {READ_TEST_CHAIN}...")

    relay_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Quote read fee
    read_gas = 150_000
    fee = relay_w3.functions.quote_read_fee(read_gas, 0).call()
    print(f"Read fee: {fee / 1e18:.6f} ETH")

    # Request block hash (0 = latest)
    request_block = 0
    print(f"Requesting block {request_block} (latest)...")

    func = relay_w3.functions.request_block_hash([], [], 0, read_gas, request_block)
    tx_hash = send_tx(w3, func, account, int(1.2 * fee))
    print(f"Transaction sent: {tx_hash.hex()}")
    print("\nWait ~30 seconds for LayerZero message to propagate...")

## 9. Test Broadcast

In [ ]:
if TEST_BROADCAST and READ_TEST_CHAIN in state_dict:
    print(f"\nTesting broadcast from {READ_TEST_CHAIN}...")

    broadcaster_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Get all configured peers
    all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
    receive_eids = []

    for eid in all_eids:
        if eid == state_dict[READ_TEST_CHAIN]["eid"] or eid == state_dict[main_chain]["eid"]:
            continue
        # Check if peer is configured
        peer = broadcaster_w3.functions.peers(eid).call()
        if int.from_bytes(peer, "big") != 0:
            receive_eids.append(eid)

    if receive_eids:
        print(f"Broadcasting to {len(receive_eids)} chains...")

        # Quote broadcast fees
        receive_gas = 150_000
        broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(
            receive_eids, receive_gas
        ).call()
        total_fee = sum(broadcast_fees)
        print(f"Total broadcast fee: {total_fee / 1e18:.6f} ETH")

        # Broadcast latest block
        func = broadcaster_w3.functions.broadcast_latest_block(
            receive_eids, broadcast_fees, receive_gas
        )
        tx_hash = send_tx(w3, func, account, total_fee)
        print(f"Broadcast transaction sent: {tx_hash.hex()}")
    else:
        print("No peers configured for broadcasting")
else:
    print("\nSkipping broadcast test")

## 10. Test Read with Broadcast

In [ ]:
if (
    TEST_BROADCAST
    and READ_TEST_CHAIN in state_dict
    and state_dict[READ_TEST_CHAIN]["read_lib"] != "unavailable"
):
    print(f"\nTesting read with broadcast from {READ_TEST_CHAIN}...")

    broadcaster_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Get receive chains
    all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
    receive_eids = []

    for eid in all_eids:
        if eid == state_dict[READ_TEST_CHAIN]["eid"] or eid == state_dict[main_chain]["eid"]:
            continue
        peer = broadcaster_w3.functions.peers(eid).call()
        if int.from_bytes(peer, "big") != 0:
            receive_eids.append(eid)

    if receive_eids:
        # Quote fees
        receive_gas = 150_000
        broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(
            receive_eids, receive_gas
        ).call()
        broadcast_fees = [int(fee * 1.01) for fee in broadcast_fees]  # Add 1% buffer

        BROADCAST_GAS = 2_000_000
        read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
            BROADCAST_GAS, sum(broadcast_fees)
        ).call()

        print(f"Read fee with broadcast: {read_fee_with_broadcast / 1e18:.6f} ETH")
        print(f"Broadcasting to {len(receive_eids)} chains...")

        # Request block hash with broadcast
        func = broadcaster_w3.functions.request_block_hash(
            receive_eids, broadcast_fees, receive_gas, BROADCAST_GAS
        )
        tx_hash = send_tx(w3, func, account, value=read_fee_with_broadcast, gas=500_000)
        print(f"Read + broadcast transaction sent: {tx_hash.hex()}")
        print(
            f"\nThis will read the latest block from {main_chain} and broadcast to all configured chains"
        )
    else:
        print("No peers configured for broadcasting")
else:
    print("\nSkipping read with broadcast test")

## 11. Submit Block Header Test

In [ ]:
# Test submitting a block header
test_chain = READ_TEST_CHAIN if READ_TEST_CHAIN in state_dict else list(state_dict.keys())[1]

if test_chain != main_chain:
    print(f"\nTesting block header submission on {test_chain}...")

    oracle_w3 = state_dict[test_chain]["oracle_w3"]
    header_verifier_w3 = state_dict[test_chain]["header_verifier_w3"]

    # Get last confirmed block
    block_number = oracle_w3.functions.last_confirmed_block_number().call()

    if block_number > 0:
        print(f"Last confirmed block: {block_number}")

        # Get current header data
        headers = oracle_w3.functions.block_header(block_number).call()
        print("\nCurrent block header data:")
        print(f"  Block hash: {headers[0].hex()}")
        print(f"  Parent hash: {headers[1].hex()}")
        print(f"  State root: {headers[2].hex()}")
        print(f"  Receipt root: {headers[3].hex()}")
        print(f"  Block number: {headers[4]}")
        print(f"  Timestamp: {headers[5]}")

        # Get block data from main chain
        main_w3 = state_dict[main_chain]["w3"]
        block_data = main_w3.eth.get_block(block_number, full_transactions=False)

        # Encode headers
        encoded_headers = encode_headers(block_data)
        print(f"\nEncoded headers: {encoded_headers.hex()[:100]}...")

        # Submit header
        try:
            w3 = state_dict[test_chain]["w3"]
            func = header_verifier_w3.functions.submit_block_header(
                oracle_w3.address, encoded_headers
            )
            tx_hash = send_tx(w3, func, account)
            print(f"\nHeader submission transaction: {tx_hash.hex()}")
        except Exception as e:
            print(f"\nError submitting header: {e}")
            if "already submitted" in str(e).lower():
                print("Header was already submitted for this block")
    else:
        print("No confirmed blocks yet - run a read test first")
else:
    print("\nSkipping header submission test on main chain")

## 12. Final State Check

In [ ]:
# Wait a bit for transactions to process
print("\nWaiting 30 seconds for transactions to process...")
time.sleep(30)

# Check final oracle state
print("\n" + "=" * 80)
print("FINAL ORACLE STATE")
print("=" * 80)

# Get main chain block number
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"\nCurrent block on {main_chain}: {main_block}")

# Check oracle state on all other chains
for chain_name in state_dict.keys():
    if chain_name == main_chain:
        continue

    oracle_w3 = state_dict[chain_name]["oracle_w3"]

    try:
        last_block = oracle_w3.functions.last_confirmed_block_number().call()
        if last_block > 0:
            block_hash = oracle_w3.functions.get_block_hash(last_block).call()
            print(f"\n{chain_name}:")
            print(f"  Last confirmed block: {last_block}")
            print(f"  Block hash: {block_hash.hex()}")
            print(f"  Blocks behind: {main_block - last_block}")

            # Check if state root is available
            try:
                state_root = oracle_w3.functions.get_state_root(last_block).call()
                if state_root != b"\x00" * 32:
                    print(f"  State root: {state_root.hex()}")
            except Exception as e:
                print(f"  Error getting state root: {e}")
                pass
        else:
            print(f"\n{chain_name}: No blocks confirmed yet")
    except Exception as e:
        print(f"\n{chain_name}: Error reading oracle state - {e}")

print("\n" + "=" * 80)
print("\nTest complete! Check LayerZero Scan for transaction details:")
print(f"https://{'testnet.' if NETWORK_TYPE == 'testnets' else ''}layerzeroscan.com/")